In [1]:
import pandas as pd 
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [2]:
# Load data from 2024
df = pd.read_csv('/Users/bennett/Desktop/trading_bot/raw_data/BTCUSDT_1h_recent.csv')

In [3]:
df.head()

,Unnamed: 0,timestamp,open,high,low,close,volume
0,0,2024-10-27 02:00:00,67121.63,67139.25,67025.73,67030.76,0.12210
1,1,2024-10-27 03:00:00,67031.14,67311.47,67030.24,67310.15,0.29002
2,2,2024-10-27 04:00:00,67310.27,67323.37,67155.66,67262.56,0.74526
3,3,2024-10-27 05:00:00,67262.08,67274.50,67160.70,67176.98,0.28337
4,4,2024-10-27 06:00:00,67179.72,67268.08,67151.35,67268.08,0.05041


# Showing default RSI

In [4]:
# Calculate RSI
df['RSI'] = ta.rsi(df['close'], length=14)

In [5]:
df.tail()

,Unnamed: 0,timestamp,open,high,low,close,volume,RSI
495,495,2024-11-16 17:00:00,90695.17,90982.11,90549.37,90982.11,0.88447,54.454781
496,496,2024-11-16 18:00:00,90854.15,91361.06,90850.85,91251.79,0.68560,57.465939
497,497,2024-11-16 19:00:00,91251.79,91360.71,90958.91,91108.11,0.51039,55.365723
498,498,2024-11-16 20:00:00,91107.64,91151.00,90687.43,90958.00,0.35168,53.179010
499,499,2024-11-16 21:00:00,90738.54,90950.81,90700.00,90772.18,0.04277,50.519059


In [6]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    subplot_titles=("Closing Price", "Relative Strength Index (RSI)"),
                    vertical_spacing=0.1)


# Add the closing price to the first row with date in the hover text
fig.add_trace(
    go.Scatter(
        x=df['timestamp'], 
        y=df['close'], 
        mode='lines+markers', 
        name='Closing Price',
        hovertemplate='Date: %{x}<br>Price: %{y:.2f}'  # Tooltip with date and price
    ),
    row=1, col=1
)

# Add the RSI to the second row with date in the hover text
fig.add_trace(
    go.Scatter(
        x=df['timestamp'], 
        y=df['RSI'], 
        mode='lines+markers', 
        name='RSI', 
        line=dict(color='orange'),
        hovertemplate='Date: %{x}<br>RSI: %{y:.2f}'  # Tooltip with date and RSI
    ),
    row=2, col=1
)

# Add horizontal lines for RSI thresholds (e.g., 30 and 70)
fig.add_shape(type="line", x0=df['timestamp'].min(), x1=df['timestamp'].max(), y0=30, y1=30,
              line=dict(color="red", dash="dash"), row=2, col=1)
fig.add_shape(type="line", x0=df['timestamp'].min(), x1=df['timestamp'].max(), y0=70, y1=70,
              line=dict(color="red", dash="dash"), row=2, col=1)

# Update layout
fig.update_layout(
    title="Closing Price and RSI with Dates",
    height=600,
    xaxis_title="Date",
    yaxis_title="Price",
    xaxis2_title="Date",
    yaxis2_title="RSI",
    template="plotly_white"
)

# Show the plot
fig.show()

# Testing Accuracy

In [7]:
def rsi_accuracy(df, rsi_length=14, rsi_oversold=30, rsi_overbought=70):
    """
    Calculate the accuracy of RSI in predicting the price movement.

    Parameters:
    - df: DataFrame with columns 'close' and 'RSI'
    - rsi_length: Length of RSI (default is 14)
    - rsi_oversold: RSI threshold for oversold (default is 30)
    - rsi_overbought: RSI threshold for overbought (default is 70)

    Returns:
    - accuracy: Float value representing the accuracy of RSI predictions,
                or None if no signals are generated.
    """
    # Generate RSI signals based on thresholds
    df['RSI_signal'] = 0  # Initialize the RSI signal column
    df.loc[df['RSI'] < rsi_oversold, 'RSI_signal'] = 1  # Buy signal
    df.loc[df['RSI'] > rsi_overbought, 'RSI_signal'] = -1  # Sell signal

    # Calculate price difference and future price
    df['price_2w_later'] = df['close'].shift(-14)
    df['price_diff'] = df['price_2w_later'] - df['close']

    # Check if signals align with price movements
    df['correct_signal'] = (
        ((df['RSI_signal'] == 1) & (df['price_diff'] > 0)) | 
        ((df['RSI_signal'] == -1) & (df['price_diff'] < 0))
    ).astype(int)

    # Filter only rows with signals
    signal_mask = df['RSI_signal'] != 0
    correct_signals = df.loc[signal_mask, 'correct_signal'].sum()
    total_signals = signal_mask.sum()

    # Calculate accuracy
    if total_signals > 0:
        accuracy = correct_signals / total_signals
    else:
        accuracy = None

    return accuracy

In [8]:
rsi_accuracy(df)

0.3018867924528302

# Optimizing Parameters

In [9]:
def optimize_rsi_period(df, min_rsi_length=3, max_rsi_length=20, rsi_oversold=30, rsi_overbought=70, min_period=3, max_period=21):
    import pandas_ta as ta

    results = {}
    profits = {}

    # Loop through different RSI lengths
    for rsi_length in range(min_rsi_length, max_rsi_length + 1):
        df['RSI'] = ta.rsi(df['close'], length=rsi_length)
        df['RSI_signal'] = 0
        df.loc[df['RSI'] < rsi_oversold, 'RSI_signal'] = 1
        df.loc[df['RSI'] > rsi_overbought, 'RSI_signal'] = -1

        # Loop through holding periods
        for period in range(min_period, max_period + 1):
            df['price_future'] = df['close'].shift(-period)
            df['price_diff'] = df['price_future'] - df['close']
            df['correct_signal'] = (
                ((df['RSI_signal'] == 1) & (df['price_diff'] > 0)) |
                ((df['RSI_signal'] == -1) & (df['price_diff'] < 0))
            ).astype(int)

            # Calculate accuracy
            signal_mask = df['RSI_signal'] != 0
            correct_signals = df.loc[signal_mask, 'correct_signal'].sum()
            total_signals = signal_mask.sum()
            accuracy = correct_signals / total_signals if total_signals > 0 else None

            # Calculate total profit
            df['profit'] = df['RSI_signal'] * df['price_diff']
            total_profit = df.loc[signal_mask, 'profit'].sum() if total_signals > 0 else None

            results[(rsi_length, period)] = accuracy
            profits[(rsi_length, period)] = total_profit

    # Find the best combination by accuracy
    best_combo = max(results, key=lambda combo: results[combo] if results[combo] is not None else -1)
    best_profit = profits[best_combo]

    return best_combo, best_profit, results, profits

In [10]:
best_combo, best_profit, results, profits = optimize_rsi_period(df)

print("Optimization Results:")
print(f"Best Combination: RSI Length = {best_combo[0]}, Holding Period = {best_combo[1]}")
print(f"Highest Accuracy: {results[best_combo]:.2%}")
print(f"Total Profit for Best Combination: ${best_profit:.2f}")

print("\nAll Tested Results:")
for key, value in results.items():
    rsi_len, period = key
    accuracy = f"{value:.2%}" if value is not None else "N/A"
    profit = f"${profits[key]:.2f}" if profits[key] is not None else "N/A"
    print(f"RSI Length: {rsi_len}, Holding Period: {period}, Accuracy: {accuracy}, Total Profit: {profit}")

Optimization Results:
Best Combination: RSI Length = 3, Holding Period = 4
Highest Accuracy: 57.38%
Total Profit for Best Combination: $-7845.29

All Tested Results:
RSI Length: 3, Holding Period: 3, Accuracy: 56.54%, Total Profit: $-5567.91
RSI Length: 3, Holding Period: 4, Accuracy: 57.38%, Total Profit: $-7845.29
RSI Length: 3, Holding Period: 5, Accuracy: 51.90%, Total Profit: $-8174.58
RSI Length: 3, Holding Period: 6, Accuracy: 50.21%, Total Profit: $-18333.54
RSI Length: 3, Holding Period: 7, Accuracy: 48.52%, Total Profit: $-23686.07
RSI Length: 3, Holding Period: 8, Accuracy: 47.26%, Total Profit: $-34576.87
RSI Length: 3, Holding Period: 9, Accuracy: 47.68%, Total Profit: $-38078.18
RSI Length: 3, Holding Period: 10, Accuracy: 45.57%, Total Profit: $-37368.39
RSI Length: 3, Holding Period: 11, Accuracy: 46.84%, Total Profit: $-41511.20
RSI Length: 3, Holding Period: 12, Accuracy: 46.41%, Total Profit: $-50305.29
RSI Length: 3, Holding Period: 13, Accuracy: 45.57%, Total Profi

- The optimal RSI length is 3 and the optimal holding period is 4 with a 57% accuracy

In [11]:
df = pd.read_csv('/Users/bennett/Desktop/trading_bot/raw_data/BTCUSDT_1h_recent.csv')

# Finding Optimal Threshholds For RSI

In [12]:
import pandas_ta as ta

def optimize_rsi_thresholds(df, rsi_length=3, holding_period=4, min_threshold=20, max_threshold=80, step=5):
    """
    Optimize RSI oversold and overbought thresholds for a given RSI length and holding period.

    Parameters:
    - df: DataFrame with a 'close' column.
    - rsi_length: RSI length to use (default is 18).
    - holding_period: Holding period to use (default is 16 days).
    - min_threshold: Minimum threshold to test (default is 20).
    - max_threshold: Maximum threshold to test (default is 80).
    - step: Increment for testing thresholds (default is 5).

    Returns:
    - best_thresholds: Tuple (oversold, overbought) with the highest accuracy.
    - results: Dictionary with thresholds as keys and values as a tuple (accuracy, total_profit).
    """
    results = {}

    # Calculate RSI with the fixed length
    df['RSI'] = ta.rsi(df['close'], length=rsi_length)

    # Loop through oversold/overbought threshold combinations
    for oversold in range(min_threshold, max_threshold, step):
        for overbought in range(oversold + step, max_threshold + 1, step):
            # Generate RSI signals based on thresholds
            df['RSI_signal'] = 0
            df.loc[df['RSI'] < oversold, 'RSI_signal'] = 1  # Buy signal
            df.loc[df['RSI'] > overbought, 'RSI_signal'] = -1  # Sell signal

            # Calculate future price and price difference
            df['price_future'] = df['close'].shift(-holding_period)
            df['price_diff'] = df['price_future'] - df['close']

            # Evaluate correctness of signals
            df['correct_signal'] = (
                ((df['RSI_signal'] == 1) & (df['price_diff'] > 0)) |
                ((df['RSI_signal'] == -1) & (df['price_diff'] < 0))
            ).astype(int)

            # Filter signals and calculate accuracy
            signal_mask = df['RSI_signal'] != 0
            correct_signals = df.loc[signal_mask, 'correct_signal'].sum()
            total_signals = signal_mask.sum()
            accuracy = correct_signals / total_signals if total_signals > 0 else None

            # Calculate total profit
            df['profit'] = df['RSI_signal'] * df['price_diff']
            total_profit = df.loc[signal_mask, 'profit'].sum() if total_signals > 0 else 0

            # Store the results
            results[(oversold, overbought)] = (accuracy, total_profit)

    # Find the best thresholds by accuracy
    best_thresholds = max(results, key=lambda t: results[t][0] if results[t][0] is not None else -1)
    return best_thresholds, results

In [13]:
# Run optimization for thresholds
best_thresholds, threshold_results = optimize_rsi_thresholds(df)

# Print the best thresholds and their results
best_accuracy, best_profit = threshold_results[best_thresholds]
print(f"Optimal Oversold/Overbought Thresholds: {best_thresholds}")
print(f"Highest Accuracy: {best_accuracy:.2%}")
print(f"Total Profit for Best Combination: ${best_profit:.2f}")

# Print all results
print("\nAll Threshold Results:")
for (oversold, overbought), (accuracy, total_profit) in threshold_results.items():
    accuracy_str = f"{accuracy:.2%}" if accuracy is not None else "N/A"
    print(f"Oversold: {oversold}, Overbought: {overbought}, Accuracy: {accuracy_str}, Total Profit: ${total_profit:.2f}")

Optimal Oversold/Overbought Thresholds: (35, 80)
Highest Accuracy: 61.43%
Total Profit for Best Combination: $20120.21

All Threshold Results:
Oversold: 20, Overbought: 25, Accuracy: 47.60%, Total Profit: $-71804.64
Oversold: 20, Overbought: 30, Accuracy: 48.68%, Total Profit: $-67843.90
Oversold: 20, Overbought: 35, Accuracy: 50.12%, Total Profit: $-57439.91
Oversold: 20, Overbought: 40, Accuracy: 50.64%, Total Profit: $-53344.83
Oversold: 20, Overbought: 45, Accuracy: 49.44%, Total Profit: $-47027.34
Oversold: 20, Overbought: 50, Accuracy: 49.40%, Total Profit: $-45621.41
Oversold: 20, Overbought: 55, Accuracy: 51.55%, Total Profit: $-27905.21
Oversold: 20, Overbought: 60, Accuracy: 52.76%, Total Profit: $-19369.66
Oversold: 20, Overbought: 65, Accuracy: 53.48%, Total Profit: $-18053.04
Oversold: 20, Overbought: 70, Accuracy: 54.12%, Total Profit: $-15591.12
Oversold: 20, Overbought: 75, Accuracy: 54.88%, Total Profit: $-5187.75
Oversold: 20, Overbought: 80, Accuracy: 56.52%, Total P

- With a RSI length and Holding Period of 3 and 4 respectively, the optimal threshold is 35 and 80

# Backtesting

In [14]:
import pandas_ta as ta

def backtest_rsi_strategy(df, rsi_length, holding_period, oversold, overbought, fee_percent=0.1):
    """
    Backtest an RSI-based trading strategy with the given parameters, including transaction fees.

    Parameters:
    - df: DataFrame with a 'close' column.
    - rsi_length: RSI length to calculate.
    - holding_period: Holding period for evaluating trades.
    - oversold: RSI oversold threshold for buy signals.
    - overbought: RSI overbought threshold for sell signals.
    - fee_percent: Transaction fee percentage per trade (default is 0.1%, or 0.001).

    Returns:
    - metrics: Dictionary with backtest results including:
        - Total profit (after fees)
        - Accuracy
        - Number of trades
        - Win rate
        - Profit per trade (after fees)
    - df: DataFrame with backtest results appended (signals, profits, etc.).
    """
    # Calculate RSI
    df['RSI'] = ta.rsi(df['close'], length=rsi_length)

    # Generate RSI signals
    df['RSI_signal'] = 0
    df.loc[df['RSI'] < oversold, 'RSI_signal'] = 1  # Buy signal
    df.loc[df['RSI'] > overbought, 'RSI_signal'] = -1  # Sell signal

    # Calculate future prices and price differences
    df['price_future'] = df['close'].shift(-holding_period)
    df['price_diff'] = df['price_future'] - df['close']

    # Calculate profits based on signals (before fees)
    df['profit'] = df['RSI_signal'] * df['price_diff']

    # Calculate transaction fees
    df['fee'] = fee_percent / 100 * df['close']  # Fee based on entry price
    df['net_profit'] = df['profit'] - df['fee']  # Profit after fees

    # Calculate correctness of signals
    df['correct_signal'] = (
        ((df['RSI_signal'] == 1) & (df['price_diff'] > 0)) |
        ((df['RSI_signal'] == -1) & (df['price_diff'] < 0))
    ).astype(int)

    # Filter valid signals
    signal_mask = df['RSI_signal'] != 0
    total_trades = signal_mask.sum()
    total_profit_after_fees = df.loc[signal_mask, 'net_profit'].sum()
    correct_signals = df.loc[signal_mask, 'correct_signal'].sum()

    # Calculate metrics
    accuracy = correct_signals / total_trades if total_trades > 0 else None
    win_rate = correct_signals / total_trades if total_trades > 0 else None
    profit_per_trade = total_profit_after_fees / total_trades if total_trades > 0 else None

    metrics = {
        "Total Profit (After Fees)": total_profit_after_fees,
        "Accuracy": accuracy,
        "Win Rate": win_rate,
        "Number of Trades": total_trades,
        "Profit per Trade (After Fees)": profit_per_trade
    }

    return metrics, df

optimal_rsi_length = 18
optimal_holding_period = 16
optimal_oversold = 30
optimal_overbought = 80

In [15]:
# Optimal parameters (from previous optimization)
optimal_rsi_length = 3
optimal_holding_period = 4
optimal_oversold = 35
optimal_overbought = 80

# Run backtest
metrics, backtested_df = backtest_rsi_strategy(
    df, 
    rsi_length=optimal_rsi_length, 
    holding_period=optimal_holding_period, 
    oversold=optimal_oversold, 
    overbought=optimal_overbought
)

# Print metrics
print("Backtest Results:")
for key, value in metrics.items():
    if value is not None:
        print(f"{key}: {value:.2f}" if isinstance(value, float) else f"{key}: {value}")
    else:
        print(f"{key}: No Trades")

Backtest Results:
Total Profit (After Fees): 4074.45
Accuracy: 0.61
Win Rate: 0.61
Number of Trades: 210
Profit per Trade (After Fees): 19.40
